# 학습된 모델 불러와서 테스트하는 코드

In [ ]:
import pennylane as qml
from pennylane import numpy as np

from tqdm import tqdm
from IPython.display import clear_output

import torch
from torch.autograd import Variable
from torchvision import datasets, transforms
from functools import reduce
import random
import math
import matplotlib.pyplot as plt
import pickle
import pandas as pd

In [ ]:
# 기본 세팅
name1 = "AAPL"
name2 = "TSLA"
cut_left = -0.1
cut_right = 0.1
num_bins = 32
num_dataset = 2000

import yfinance as yf
import datetime

def get_stocks_datas(ticker_names: list[str], start_date, end_date):
    # WARN: returns는 prices보다 길이가 1 작다.
    prices = {}
    returns = {}
    for name in ticker_names:
        data = yf.Ticker(name)
        data = data.history(start=start_date, end=end_date, interval="1d")
        prices[name] = data['Close']
        returns[name] = data['Close'].pct_change()[1:]
    return prices, returns

start_date = datetime.datetime(2011, 1, 1)
end_date = datetime.datetime(2022, 12, 31)
stock_prices, stock_returns = get_stocks_datas([name1, name2], start_date, end_date)

In [ ]:
# 산점도 그리기
plt.scatter(stock_returns[name1].values, stock_returns[name2].values, alpha=0.5)
plt.title(f'{name1}/{name2} daily returns')
plt.xlabel(name1)
plt.ylabel(name2)
plt.xlim(-0.15, 0.15)
plt.ylim(-0.15, 0.15)
plt.grid(True)
plt.show()

In [ ]:
x = []
y = []
c = []
for k in np.linspace(0, 1, 100):
    merged_returns = k * stock_returns[name1].values + (1 - k) * stock_returns[name2].values
    x.append(merged_returns.std())
    y.append(merged_returns.mean())
    c.append(k)

plt.scatter(x, y, c=c, cmap='viridis')
plt.xlabel('Standard Deviation')
plt.ylabel('Mean')
plt.title(f'std-mean plot {name1}/{name2}')
plt.colorbar()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

bin_edges = np.linspace(cut_left, cut_right, num_bins+1)

plt.figure(figsize=(10, 6))

plt.hist(stock_returns[name1].values, bins=bin_edges, alpha=0.7, label=name1)
plt.hist(stock_returns[name2].values, bins=bin_edges, alpha=0.7, label=name2)

plt.title(f'Distribution of Daily Returns: {name1} vs. {name2}')
plt.xlabel('Daily Return')
plt.ylabel('Frequency')
plt.legend()
plt.show()

print(f"Mean of {name1}: {stock_returns[name1].mean():.4f}")
print(f"Mean of {name2}: {stock_returns[name2].mean():.4f}")


In [ ]:
# setting torch device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# 이 세팅들은 param.txt에서 참고하자
noise_qubits = 5
code_qubits = 1
output_qubits = 5 # 실제로 출력 상태를 만드는데 쓰이는 개수
n_qubits = 5

n_layers = 40
BATCH_SIZE = 16

print("n_qubits = {} n_layers = {}".format(n_qubits, n_layers))
dev = qml.device("lightning.qubit", wires=n_qubits) # 제너레이터 돌려서 값 뽑아내는데 쓰임. 이중 몇개 골라서 판별자도 학습함.

In [ ]:
def generator_init(generator_input):
    for i in range(n_qubits):
        qml.RY(generator_input[i]*np.pi/2, wires=i) # TODO: *a 해서 값 범위 맞추기

def generator_layer(params):
    for i in range(n_qubits):
        qml.RX(params[i][0], wires=i)
        qml.RY(params[i][1], wires=i)
        qml.RZ(params[i][2], wires=i)
    
    for i in range(n_qubits):
        qml.CNOT(wires=[i, (i+1)%n_qubits])

@qml.qnode(dev, interface="torch")
def generator_circuit(params, generator_input):
    generator_init(generator_input)
    for param in params:
        generator_layer(param)
    return qml.probs(wires=range(output_qubits))


def generator_forward(params, generator_input):
    generator_output = [generator_circuit(params, single_in) for single_in in generator_input] # (BATCH_SIZE, 2**output_qubits) 차원
    generator_output = torch.stack(generator_output) # (BATCH_SIZE, 2**output_qubits) 차원
    return generator_output

In [ ]:
import torch.nn as nn
from torch.nn import functional as F

class LinearDiscriminator(nn.Module):
    def __init__(self, input_dim=num_bins):
        super().__init__()
        self.hidden_size = 100
        self.layers = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm1d(self.hidden_size), 
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size*16, self.hidden_size),
            nn.BatchNorm1d(self.hidden_size), 
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.BatchNorm1d(self.hidden_size), 
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, 1),
            nn.Sigmoid())

        self.input_dim = input_dim

    def forward(self, x):
        return self.layers(x)


class LinearMine(nn.Module):
    def __init__(self):
        super(LinearMine, self).__init__()
        H = 200
        self.fc1 = nn.Linear(code_qubits, H)
        self.fc2 = nn.Linear(2**output_qubits, H)
        self.fc3 = nn.Linear(H, 1)

    def forward(self, x, y):
        h1 = F.relu(self.fc1(x)+self.fc2(y))
        h2 = self.fc3(h1)
        return h2
        

In [ ]:
generator_params = Variable(torch.tensor(np.random.normal(-np.pi, np.pi, (n_layers, n_qubits, 3))), requires_grad=True)
print("parameter shape: ", generator_params.shape)
discriminator = LinearDiscriminator()
mine = LinearMine()

## 모델 불러오기

In [ ]:
import pickle
file_num = 12
use_mine = True
title = f'mine={use_mine}_try={file_num}'

load_epoch = 165

with open(f'result/{title}/discriminator_{load_epoch}.pkl', 'rb') as file:
    discriminator = pickle.load(file)

with open(f'result/{title}/generator_{load_epoch}.pkl', 'rb') as file:
    generator_params = pickle.load(file)

In [ ]:
import warnings
def visualize_output(gen_outputs, gen_codes):
    warnings.simplefilter("ignore")


    random_indices = np.random.choice(gen_outputs.shape[0], size=9, replace=False)
    selected_samples = [gen_outputs[i] for i in random_indices]
    selected_codes = [gen_codes[i] for i in random_indices]

    sorted_indices = np.argsort(selected_codes)
    sorted_selected_codes = [selected_codes[i] for i in sorted_indices]
    sorted_selected_samples = [selected_samples[i] for i in sorted_indices]

    plt.figure(figsize=(12, 9))

    for i, sample in enumerate(sorted_selected_samples):
        plt.subplot(3, 3, i + 1)
        plt.bar(np.linspace(-0.1, 0.1, num_bins), sample, width=0.2*(num_bins+1)/num_bins**2)
        plt.title(f'{sorted_selected_codes[i]:.3f}')
        plt.xlabel('Return')
        plt.ylabel('Frequency')

    plt.tight_layout()
    plt.suptitle(f"random imgs")
    plt.show()

In [ ]:
def get_averages(codes: list[float], shots=128, code_ind: int = -1):
    returns = []
    for code in tqdm(codes):
        seed = torch.rand((shots, n_qubits))
        seed[:, code_ind] = code
        result = generator_forward(generator_params, seed).detach().numpy()
        returns.append(result.mean(axis=0))
    returns = np.array(returns)
    return returns


def visualize_average(shots=128):
    # 3*3 그래프와 함께 9개점 plot std-mean 그림
    plt.figure(figsize=(12, 8))
    x = []
    y = []
    c = []

    codes = np.linspace(0, 1, 9)
    probs_list = get_averages(codes, shots=shots)
    for i in tqdm(range(len(codes))):
        code = codes[i]
        probs = probs_list[i]
        plt.subplot(3, 3, i + 1)
        plt.bar(np.linspace(-0.1, 0.1, num_bins), probs, width=0.2*(num_bins+1)/num_bins**2)
        plt.title(f'{code:.3f}')
        plt.ylim(0, 0.2)
        plt.xlabel('Return')
        plt.ylabel('Frequency')
        
        d = (cut_right - cut_left) / num_bins * (num_bins // 2 - 0.5)
        vals = np.linspace(-d, d, num_bins)

        avg = np.sum(probs * vals)
        std = np.sqrt(np.sum(probs * (vals - avg)**2))
        x.append(std)
        y.append(avg)
        c.append(code.item())

    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.suptitle(f"mine={use_mine} epoch={load_epoch}")
    plt.savefig(f'result/{title}/analize_{load_epoch}_1.png')
    plt.show()

    plt.figure(figsize=(5, 6))
    plt.scatter(x, y, c=c, cmap='viridis')
    plt.plot(x, y)
    plt.xlabel('Standard Deviation')
    plt.ylabel('Mean')
    plt.title(f'std-mean plot')
    plt.colorbar()
    plt.savefig(f'result/{title}/analize_{load_epoch}_2.png')
    plt.show()

def visualize_std_mean(codes: list[float], shots=128):
    # std-mean 공간을 plot하기 위한 함수
    probs_list = get_averages(codes, shots=shots)
    x = []
    y = []
    c = []
    for i in tqdm(range(len(codes))):
        code = codes[i]
        probs = probs_list[i]

        d = (cut_right - cut_left) / num_bins * (num_bins // 2 - 0.5)
        vals = np.linspace(-d, d, num_bins)

        avg = np.sum(probs * vals)
        std = np.sqrt(np.sum(probs * (vals - avg)**2))
        x.append(std)
        y.append(avg)
        c.append(code.item())
    
    plt.figure(figsize=(5, 6))
    plt.scatter(x, y, c=c, cmap='viridis')
    plt.plot(x, y)
    plt.xlabel('Standard Deviation')
    plt.ylabel('Mean')
    plt.title(f'std-mean plot {name1}/{name2}')
    plt.colorbar()
    plt.savefig(f'result/{title}/analize_{load_epoch}_3({shots}).png')
    plt.show()

def visualize_variables(codes: list[float], shots=128):
    # 모든 노이즈들의 자취를 std-mean 공간에 plot하기 위한 함수
    markerstyles = ['o', '*', '+', 'v', '^']
    linestyles = ['-', '--', '-.', ':', '-']
    labels = ['noise1', 'noise2', 'noise3', 'noise4', 'code']

    plt.figure(figsize=(5, 6))
    xs = []
    ys = []
    cs = []
    for v in range(n_qubits):
        probs_list = get_averages(codes, shots=shots, code_ind=v) # 모든 위치를 테스트해봄
        x = []
        y = []
        c = []
        for i in tqdm(range(len(codes))):
            code = codes[i]
            probs = probs_list[i]

            d = (cut_right - cut_left) / num_bins * (num_bins // 2 - 0.5)
            vals = np.linspace(-d, d, num_bins)

            avg = np.sum(probs * vals)
            std = np.sqrt(np.sum(probs * (vals - avg)**2))
            x.append(std)
            y.append(avg)
            c.append(code.item())
    
        plt.scatter(x, y, c=c, cmap='viridis', marker=markerstyles[v])
        plt.plot(x, y, linestyle='--', label=labels[v])
        xs.append(x)
        ys.append(y)
        cs.append(c)
        
    plt.xlabel('Standard Deviation')
    plt.ylabel('Mean')
    plt.legend()
    plt.title(f'std-mean plot of codes {name1}/{name2}')
    plt.colorbar()
    plt.savefig(f'result/{title}/analize_{load_epoch}_4({shots}).png')
    plt.show()
    return xs, ys, cs

In [ ]:
visualize_average(shots=128)

In [ ]:
visualize_std_mean(np.linspace(0, 1, 20), shots=256)

In [31]:
shots = 256
xs, ys, cs = visualize_variables(np.linspace(0, 1, 10), shots=shots)

 10%|█         | 1/10 [00:06<00:57,  6.34s/it]

In [ ]:
markerstyles = ['o', '*', '+', 'v', '^']
labels = ['noise1', 'noise2', 'noise3', 'noise4', 'code']

for v in range(n_qubits):
    plt.figure(figsize=(5, 6))
    x = xs[v]
    y = ys[v]
    c = cs[v]

    plt.scatter(x, y, c=c, cmap='viridis', marker=markerstyles[v])
    plt.plot(x, y, label=labels[v])
    xs.append(x)
    ys.append(y)
    cs.append(c)
    
    plt.xlabel('Standard Deviation')
    plt.ylabel('Mean')
    plt.legend()
    plt.title(f'std-mean plot {name1}/{name2}')
    plt.colorbar()
    plt.savefig(f'result/{title}/analize_{load_epoch}_v{v}({shots}).png')
    plt.show()